# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [47]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from pandas.io.json import json_normalize
import copy 

In [48]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb")
cur = conn.cursor()

In [46]:
conn.close()

In [49]:
conn.set_session(autocommit=True)

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
#uses the get_files function to get a list of the song files
song_files = get_files('data/song_data')

In [7]:
#select first song in the list
song_files[0]

'/Users/johnrick/Desktop/udacity/project_template/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [8]:
#see what's in each json file
pd.read_json(song_files[3], lines=True)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0


In [9]:
#create a dataframe, adds each json file to dataframe
df = pd.DataFrame()
for song in range(len(song_files)):
    dfItem = pd.read_json(song_files[song], lines=True)
    df = df.append(dfItem, ignore_index = True)

In [10]:
df.head(2)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
#select columns: song_id, title, artist_id, year, duration
['song_id', 'title', 'artist_id', 'year', 'duration']

In [11]:
df[['song_id', 'title', 'artist_id', 'year', 'duration']].loc[0]

song_id      SONHOTT12A8C13493C
title           Something Girls
artist_id    AR7G5I41187FB4CE6C
year                       1982
duration                233.404
Name: 0, dtype: object

In [12]:
#use df.values to select values
df[['song_id', 'title', 'artist_id', 'year', 'duration']].loc[0].values

array(['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C',
       1982, 233.40363], dtype=object)

In [22]:
df[['song_id', 'title', 'artist_id', 'year', 'duration']].loc[0].to_list()

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [23]:
# just getting the first song. Use tolist()
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].loc[0].tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [24]:
type(song_data)

list

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [50]:
cur.execute(song_table_insert, song_data)
conn.commit()

ProgrammingError: can't adapt type 'numpy.int64'

In [51]:
cur.execute('SELECT * FROM songs;')
conn.commit()

UndefinedTable: relation "songs" does not exist
LINE 1: SELECT * FROM songs;
                      ^


In [32]:
song_data[2]

'AR7G5I41187FB4CE6C'

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [102]:
# Select columns for artist ID, name, location, latitude, and longitude
['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

['artist_id',
 'artist_name',
 'artist_location',
 'artist_latitude',
 'artist_longitude']

In [103]:
# Use df.values to select just the values from the dataframe
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
2,ARXR32B1187FB57099,Gob,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955
...,...,...,...,...,...
66,ARAGB2O1187FB3A161,Pucho & His Latin Soul Brothers,,NaN,NaN
67,AR62SOJ1187FB47BB5,Chase & Status,,NaN,NaN
68,ARBEBBY1187B9B43DB,Tom Petty,"Gainesville, FL",NaN,NaN
69,ARNNKDK1187B98BBD5,Jinx,Zagreb Croatia,45.80726,15.96760


In [109]:
# Index to select the first (only) record in the dataframe
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].loc[0]

artist_id           AR7G5I41187FB4CE6C
artist_name                   Adam Ant
artist_location        London, England
artist_latitude                    NaN
artist_longitude                   NaN
Name: 0, dtype: object

In [111]:
#test other row to see if there are missing values
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].iloc[:3]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
2,ARXR32B1187FB57099,Gob,,NaN,NaN


In [112]:
# save artist data
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].iloc[0]
artist_data

artist_id           AR7G5I41187FB4CE6C
artist_name                   Adam Ant
artist_location        London, England
artist_latitude                    NaN
artist_longitude                   NaN
Name: 0, dtype: object

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [86]:
#use the get_files function to get a list of the json files
log_files = get_files('data/log_data')

In [89]:
song_play_test = pd.read_json(log_files[0], lines=True)

In [118]:
#confirm that each json file has different sizes
pd.read_json(log_files[1], lines=True).shape

(299, 18)

In [244]:
pd.read_json(log_files[0], lines=True).columns == pd.read_json(log_files[2], lines=True).columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [121]:
#put everything into one dataframe
song_play_test = pd.DataFrame()
for i in range(len(log_files)):
    dfItem = pd.read_json(log_files[i], lines=True)
    song_play_test = song_play_test.append(dfItem)
    

In [246]:
#"index column" needs to be dropped and 
song_play_test.tail()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
8051,353,Motion City Soundtrack,Logged In,Stefany,F,1,White,205.26975,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,Fell In Love Without You (Acoustic),200,1543092353796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
8052,354,Ticanaf,Logged In,Stefany,F,2,White,2594.87302,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,The Thousand Names of Lord Shiva (Part 1),200,1543092558796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
8053,355,The Far East Movement,Logged In,Ryan,M,0,Smith,228.20526,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,Dance Like Michael Jackson,200,1543097750796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
8054,356,the bird and the bee,Logged In,Ryan,M,1,Smith,189.51791,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,You're A Cad,200,1543097978796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
8055,357,Shakira,Logged In,Christian,F,0,Porter,193.82812,free,"Elkhart-Goshen, IN",PUT,NextSong,1.540897e+12,366,Inevitable,200,1543103174796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",11


In [133]:
song_play_test.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [136]:
#reseting index because we have included many indeces of the same name
song_play_test = song_play_test.reset_index()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [142]:
# Filter records by NextSong action in the page column
time_df = song_play_test[song_play_test['page'] =='NextSong']

In [149]:
#convert ts column to date time
time_df.ts = pd.to_datetime(time_df.ts, unit='ms')

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order

Hint: use pandas' dt attribute to access easily datetimelike properties.

Specify labels for these columns and set to column_labels

Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [154]:
#extract timestamp, hour, day
time_df['ts'].dt.hour

0        2
1        2
2        2
4        4
5        4
        ..
8051    20
8052    20
8053    22
8054    22
8055    23
Name: ts, Length: 6820, dtype: int64

In [ ]:
# different .dt functions 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.dayofweek.html
    

In [189]:
#create an empty dataframe
time_df_final = pd.DataFrame()

In [216]:
#save all the relevent columns into a list
cols = [time_df.ts.dt.time, time_df.ts.dt.hour,time_df.ts.dt.day, time_df.ts.dt.weekofyear,time_df.ts.dt.month,time_df.ts.dt.year, time_df.ts.dt.weekday]

In [232]:
#name of columns, must be the same as the order of list of time_data
df_time_columns = ['time', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

In [239]:
# create dataframe with each column saved
for i in range(len(df_time_columns)):
    time_df_final[df_time_columns[i]] = cols[i]

In [249]:
time_df_final.head()

,time,hour,day,weekofyear,month,year,weekday
0,02:33:56.796000,2,11,45,11,2018,6
1,02:36:10.796000,2,11,45,11,2018,6
2,02:40:34.796000,2,11,45,11,2018,6
4,04:36:13.796000,4,11,45,11,2018,6
5,04:36:46.796000,4,11,45,11,2018,6


In [233]:
df_time_columns

['time', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

In [188]:
[time_df.ts.dt.time, time_df.ts.dt.hour,time_df.ts.dt.day, time_df.ts.dt.weekofyear,time_df.ts.dt.month,time_df.ts.dt.year, time_df.ts.dt.weekday]

[[0       02:33:56.796000
  1       02:36:10.796000
  2       02:40:34.796000
  4       04:36:13.796000
  5       04:36:46.796000
               ...       
  8051    20:45:53.796000
  8052    20:49:18.796000
  8053    22:15:50.796000
  8054    22:19:38.796000
  8055    23:46:14.796000
  Name: ts, Length: 6820, dtype: object, 0        2
  1        2
  2        2
  4        4
  5        4
          ..
  8051    20
  8052    20
  8053    22
  8054    22
  8055    23
  Name: ts, Length: 6820, dtype: int64, 0       11
  1       11
  2       11
  4       11
  5       11
          ..
  8051    24
  8052    24
  8053    24
  8054    24
  8055    24
  Name: ts, Length: 6820, dtype: int64, 0       45
  1       45
  2       45
  4       45
  5       45
          ..
  8051    47
  8052    47
  8053    47
  8054    47
  8055    47
  Name: ts, Length: 6820, dtype: int64, 0       11
  1       11
  2       11
  4       11
  5       11
          ..
  8051    11
  8052    11
  8053    11
  8054    11
  

In [252]:
# easier way to create dataframe from transformaions (uses transpose)
time_df_2 = pd.DataFrame([time_df.ts.dt.time, time_df.ts.dt.hour,time_df.ts.dt.day, time_df.ts.dt.weekofyear,time_df.ts.dt.month,time_df.ts.dt.year, time_df.ts.dt.weekday]).T

In [254]:
time_df_2.columns

Index(['ts', 'ts', 'ts', 'ts', 'ts', 'ts', 'ts'], dtype='object')

In [256]:
df_time_columns

['time', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

In [261]:
# rename columns
time_df_2.columns = df_time_columns

In [262]:
time_df_2.head()

,time,hour,day,weekofyear,month,year,weekday
0,02:33:56.796000,2,11,45,11,2018,6
1,02:36:10.796000,2,11,45,11,2018,6
2,02:40:34.796000,2,11,45,11,2018,6
4,04:36:13.796000,4,11,45,11,2018,6
5,04:36:46.796000,4,11,45,11,2018,6


In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [264]:
# Select columns for user ID, first name, last name, gender and level and set to user_df

In [267]:
user_df_cols = ['userId', 'firstName', 'lastName', 'gender', 'level']

In [265]:
song_play_test.shape

(8056, 19)

In [273]:
user_df_cols

['userId', 'firstName', 'lastName', 'gender', 'level']

In [280]:
from copy import deepcopy

In [287]:
user_df.dtypes

userId       object
firstName    object
lastName     object
gender       object
level        object
dtype: object

In [290]:
type(user_df['userId'][0])

str

In [282]:
user_df = deepcopy(song_play_test[user_df_cols])

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.